## Pressure Poisson Equation
In this notebook, we show your examples of directly solving Navier-Stokes equations in the velocity space. To do this, we need to derive the pressure Poisson equation from the Navier-Stokes equations. Given the Navier-Stokes equations
$$
\begin{align}
\frac{\partial\mathbf{u}}{\partial t}+\left(\mathbf{u}\cdot\nabla\right)\mathbf{u}&=-\nabla p+\nu \nabla^2 \mathbf{u} + \mathbf{f} ,\\
\nabla\cdot\mathbf{u}&=0,
\end{align}
$$

we could take the divergence of the first equation and use the second equation to eliminate some terms:

* The divergence of the time derivative term:
$$
\nabla\cdot\frac{\partial\mathbf{u}}{\partial t}=\frac{\partial}{\partial t}\left(\nabla\cdot\mathbf{u}\right)=0
$$

* The divergence of the pressure gradient term:

$$
\nabla \cdot \nabla p = \nabla^2 p
$$

* The divergence of the viscous term:

$$
\nabla \cdot \nu \nabla^2 \mathbf{u} = \nu \nabla^2 \left(\nabla\cdot\mathbf{u}\right) = 0
$$

The convection term and external force term are not divergence-free, so we keep them. Thus, we have
$$
\nabla \cdot (\left(\mathbf{u}\cdot\nabla\right)\mathbf{u}-f) = -\nabla^2 p
$$


In [1]:
from torchfsm.operator import Laplacian,Operator,NSPressureConvection

def NavierStokes(nu:float)->Operator:
    return NSPressureConvection()+nu*Laplacian()

# TODO: implement a standard integrator
# Value Range Doesn not work

In [ ]:
from torchfsm.mesh import MeshGrid
from torchfsm.traj_recorder import CPURecorder,IntervalController
import numpy as np
import torch

mesh=MeshGrid([(0,2*np.pi,128),
               (0,2*np.pi,128),
               (0,2*np.pi,128)],
               device='cuda')
x,y,z=mesh.bc_mesh_grid()
u=torch.cat(
    [torch.sin(x)*torch.cos(y)*torch.cos(z),
     -torch.cos(x)*torch.sin(y)*torch.cos(z),
     torch.zeros_like(x)],
    dim=1)
ns=NavierStokes(1./1600)
traj=ns.integrate(
    u,
    dt=0.001,
    step=10000,
    mesh=mesh,
    trajectory_recorder=CPURecorder(control_func=IntervalController(200)),
    progressive=True
)

Integrating:   0%|          | 0/10000 [00:00<?, ?it/s]

In [3]:
from torchfsm.plot import plot_traj
plot_traj(traj)